In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

2023-08-12 13:29:34.041534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 13:29:35.387838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#define the convnet 
class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = models.Sequential()
        # CONV => RELU => POOL
        model.add(layers.Convolution2D(20, (5, 5), activation='relu', input_shape=input_shape))
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # CONV => RELU => POOL
        model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # Flatten => RELU layers
        model.add(layers.Flatten())
        model.add(layers.Dense(500, activation='relu'))
        # a softmax classifier
        model.add(layers.Dense(classes, activation="softmax"))
        return model

In [5]:
# network and training
EPOCHS = 5
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT=0.90

IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
NB_CLASSES = 10  # number of outputs = number of digits

In [6]:
# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [7]:
# reshape
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

In [8]:
# normalize
X_train, X_test = X_train / 255.0, X_test / 255.0

In [9]:
# cast
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [10]:
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [11]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, NB_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NB_CLASSES)

In [12]:
# initialize the optimizer and model
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 50)          25050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 50)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 500)               4

In [13]:
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])

In [14]:
# use TensorBoard, princess Aurora!
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [15]:
# fit 
history = model.fit(X_train, y_train, 
                    batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=VERBOSE, validation_split=VALIDATION_SPLIT,
                    callbacks=callbacks)

Epoch 1/5
47/47 [==============================] - ETA: 0s - loss: 0.7978 - accuracy: 0.7611

2023-08-12 13:33:26.551962: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169347136 exceeds 10% of free system memory.


47/47 [==============================] - 7s 138ms/step - loss: 0.7978 - accuracy: 0.7611 - val_loss: 0.3206 - val_accuracy: 0.9023
Epoch 2/5
47/47 [==============================] - 6s 126ms/step - loss: 0.2209 - accuracy: 0.9375 - val_loss: 0.2037 - val_accuracy: 0.9359
Epoch 3/5
47/47 [==============================] - 6s 126ms/step - loss: 0.1380 - accuracy: 0.9632 - val_loss: 0.1435 - val_accuracy: 0.9555
Epoch 4/5
47/47 [==============================] - 6s 126ms/step - loss: 0.1048 - accuracy: 0.9680 - val_loss: 0.1201 - val_accuracy: 0.9630
Epoch 5/5
47/47 [==============================] - 6s 127ms/step - loss: 0.0753 - accuracy: 0.9772 - val_loss: 0.1046 - val_accuracy: 0.9676


In [16]:
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 2s 7ms/step - loss: 0.0875 - accuracy: 0.9715

Test score: 0.08754158020019531
Test accuracy: 0.9714999794960022


In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [18]:
%tensorboard --logdir logs